### Part 4 L2 Regularization + Flip and blur the test set images


In [23]:
import numpy as np
import keras
from keras.datasets import mnist 
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [24]:
def resizing(mnist):
     train_data = []
     for img in mnist:
            resized_img = cv2.resize(img, (32, 32))
            train_data.append(resized_img)
     return train_data

def flip_ver(mnist):
     train_data = []
     for img in mnist:
            resized_img = cv2.resize(img, (32, 32))
            resized_img = cv2.flip(resized_img, 0)
            train_data.append(resized_img)
     return train_data

def flip_hor(mnist):
     train_data = []
     for img in mnist:
            resized_img = cv2.resize(img, (32, 32))
            resized_img = cv2.flip(resized_img, 1)
            train_data.append(resized_img)
     return train_data
verX_train = np.array(flip_ver(X_train))
horX_train = np.array(flip_hor(X_train))
verX_test = np.array(flip_ver(X_test))
horX_test = np.array(flip_hor(X_test))

In [25]:
from keras import backend
if keras.backend.image_data_format() == 'channels_first':
  verX_train = verX_train.reshape(len(verX_train), 1, 32, 32)
  verX_test = verX_test.reshape(len(verX_test), 1, 32, 32)
  horX_train = horX_train.reshape(len(horX_train), 1, 32, 32)
  horX_test = horX_test.reshape(len(horX_test), 1, 32, 32)
  input_shape = (1, 32, 32)
else:
  verX_train = verX_train.reshape(len(verX_train), 32, 32, 1)
  verX_test = verX_test.reshape(len(verX_test), 32, 32, 1)
  horX_train = horX_train.reshape(len(horX_train), 32, 32, 1)
  horX_test = horX_test.reshape(len(horX_test), 32, 32, 1)
  input_shape = (32, 32, 1)

In [26]:
from keras.utils import np_utils
def nor(dataset):
  dataset = dataset.astype('float32')
  dataset /= 255
  return dataset
verX_test = nor(verX_test)
horX_test = nor(horX_test)
verX_train = nor(verX_train)
horX_train = nor(horX_train)
Y_train = keras.utils.np_utils.to_categorical(Y_train, 10)
Y_test = keras.utils.np_utils.to_categorical(Y_test, 10)

In [27]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten 
from keras import regularizers
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, padding='same', kernel_regularizer=regularizers.l2(0.001)))
model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', input_shape=input_shape, padding='same', kernel_regularizer=regularizers.l2(0.001)))
model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu', input_shape=input_shape, padding='same', kernel_regularizer=regularizers.l2(0.001)))
model.add(Conv2D(256, (3, 3), activation='relu', input_shape=input_shape, padding='same', kernel_regularizer=regularizers.l2(0.001)))
model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same', kernel_regularizer=regularizers.l2(0.001)))
model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same', kernel_regularizer=regularizers.l2(0.001)))
model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same', kernel_regularizer=regularizers.l2(0.001)))
model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same', kernel_regularizer=regularizers.l2(0.001)))
model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
model.add(Flatten()) 
model.add(Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax', kernel_regularizer=regularizers.l2(0.001)))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 32, 32, 64)        640       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 16, 16, 64)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 16, 16, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 8, 8, 128)        0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 8, 8, 256)         295168    
                                                                 
 conv2d_11 (Conv2D)          (None, 8, 8, 256)        

In [28]:
from tensorflow.keras.optimizers import SGD
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [31]:
history = model.fit(verX_train[:6000], Y_train[:6000], batch_size=32, validation_data=(verX_test[:1000],Y_test[:1000]), epochs=10, verbose=1)

Epoch 1/10
188/188 [==============================] - 247s 1s/step - loss: 8.9239 - accuracy: 0.1593 - val_loss: 8.5978 - val_accuracy: 0.2630
Epoch 2/10
188/188 [==============================] - 248s 1s/step - loss: 8.4418 - accuracy: 0.1522 - val_loss: 8.1964 - val_accuracy: 0.1590
Epoch 3/10
188/188 [==============================] - 249s 1s/step - loss: 7.4932 - accuracy: 0.3493 - val_loss: 6.6459 - val_accuracy: 0.5620
Epoch 4/10
188/188 [==============================] - 247s 1s/step - loss: 5.9375 - accuracy: 0.7962 - val_loss: 5.8181 - val_accuracy: 0.8120
Epoch 5/10
188/188 [==============================] - 247s 1s/step - loss: 5.2266 - accuracy: 0.9257 - val_loss: 5.0553 - val_accuracy: 0.9270
Epoch 6/10
188/188 [==============================] - 246s 1s/step - loss: 4.7989 - accuracy: 0.9547 - val_loss: 4.6755 - val_accuracy: 0.9390
Epoch 7/10
188/188 [==============================] - 246s 1s/step - loss: 4.4463 - accuracy: 0.9657 - val_loss: 4.3356 - val_accuracy: 0.9520

ValueError: ignored

In [32]:
model.evaluate(verX_test[:1000], Y_test[:1000], batch_size=32)[1]

32/32 [==============================] - 5s 166ms/step - loss: 3.5043 - accuracy: 0.9600


0.9599999785423279

##### The vertical flip test accuracy with regularization is 0.96.

In [34]:
history = model.fit(horX_train[:6000], Y_train[:6000], batch_size=32, validation_data=(horX_test[:1000],Y_test[:1000]), epochs=10, verbose=1)
model.evaluate(horX_test[:1000], Y_test[:1000], batch_size=32)

Epoch 1/10
188/188 [==============================] - 255s 1s/step - loss: 3.7978 - accuracy: 0.8303 - val_loss: 3.3495 - val_accuracy: 0.9490
Epoch 2/10
188/188 [==============================] - 246s 1s/step - loss: 3.1815 - accuracy: 0.9608 - val_loss: 3.0517 - val_accuracy: 0.9620
Epoch 3/10
188/188 [==============================] - 247s 1s/step - loss: 2.9290 - accuracy: 0.9700 - val_loss: 2.8620 - val_accuracy: 0.9610
Epoch 4/10
188/188 [==============================] - 244s 1s/step - loss: 2.7069 - accuracy: 0.9817 - val_loss: 2.6808 - val_accuracy: 0.9590
Epoch 5/10
188/188 [==============================] - 244s 1s/step - loss: 2.5302 - accuracy: 0.9787 - val_loss: 2.4684 - val_accuracy: 0.9640
Epoch 6/10
188/188 [==============================] - 246s 1s/step - loss: 2.3384 - accuracy: 0.9873 - val_loss: 2.3096 - val_accuracy: 0.9670
Epoch 7/10
188/188 [==============================] - 247s 1s/step - loss: 2.1855 - accuracy: 0.9833 - val_loss: 2.3477 - val_accuracy: 0.9190

[1.7995318174362183, 0.9629999995231628]

##### The horizontal flip test accuracy with regularization is 0.9630.

### Add standard Gaussian noise

In [38]:
import numpy as np
import keras
from keras.datasets import mnist 
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [39]:
def resizing(mnist):
     train_data = []
     for img in mnist:
            resized_img = cv2.resize(img, (32, 32))
            train_data.append(resized_img)
     return train_data

newX_train = np.array(resizing(X_train))
newX_test = np.array(resizing(X_test))

In [40]:
from keras import backend
if keras.backend.image_data_format() == 'channels_first':
  newX_train = newX_train.reshape(len(newX_train), 1, 32, 32)
  newX_test = newX_test.reshape(len(newX_test), 1, 32, 32)
  input_shape = (1, 32, 32)
else:
  newX_train = newX_train.reshape(len(newX_train), 32, 32, 1)
  newX_test = newX_test.reshape(len(newX_test), 32, 32, 1)
  input_shape = (32, 32, 1)

In [41]:
from keras.utils import np_utils
newX_train = newX_train.astype('float32')
newX_train /= 255
newX_test = newX_test.astype('float32')
newX_test /= 255
Y_train = keras.utils.np_utils.to_categorical(Y_train, 10)
Y_test = keras.utils.np_utils.to_categorical(Y_test, 10)

In [43]:
from tensorflow.keras.layers import GaussianNoise
var = [0.01, 0.1, 1]
for i in var:
  sample = GaussianNoise(i, dtype=tf.float64)
  noisey = sample(newX_test,training=True) 
  model = Sequential()
  model.add(Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
  model.add(Conv2D(128, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
  model.add(Conv2D(256, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(Conv2D(256, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
  model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
  model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
  model.add(Flatten()) 
  model.add(Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
  model.add(Dropout(0.5))
  model.add(Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax', kernel_regularizer=regularizers.l2(0.001)))
  sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
  model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
  history = model.fit(newX_train[:6000], Y_train[:6000], batch_size=32, validation_data=(newX_test[:1000],Y_test[:1000]), epochs=10, verbose=1)
  acc = model.evaluate(newX_test[:1000], Y_test[:1000], batch_size=32)[1]
  print("Test accuracy when variance is " + str(i) + " is " + str(acc))


Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


188/188 [==============================] - 278s 1s/step - loss: 7.1589 - accuracy: 0.1122 - val_loss: 6.9753 - val_accuracy: 0.1260
Epoch 2/10
188/188 [==============================] - 271s 1s/step - loss: 6.7670 - accuracy: 0.1900 - val_loss: 6.2825 - val_accuracy: 0.2890
Epoch 3/10
188/188 [==============================] - 272s 1s/step - loss: 5.3112 - accuracy: 0.6085 - val_loss: 4.5182 - val_accuracy: 0.8610
Epoch 4/10
188/188 [==============================] - 272s 1s/step - loss: 4.1948 - accuracy: 0.9173 - val_loss: 4.0191 - val_accuracy: 0.9210
Epoch 5/10
188/188 [==============================] - 274s 1s/step - loss: 3.7826 - accuracy: 0.9520 - val_loss: 3.6751 - val_accuracy: 0.9490
Epoch 6/10
188/188 [==============================] - 276s 1s/step - loss: 3.4707 - accuracy: 0.9702 - val_loss: 3.3926 - val_accuracy: 0.9550
Epoch 7/10
188/188 [==============================] - 278s 1s/step - loss: 3.2086 - accuracy: 0.9748 - val_loss: 3.1632 - val_accuracy: 0.9450
Epoch 8/10

#### Summary

1.   Test Accuracy when Gaussian noise of variance 0.01: 0.962
2.   Test Accuracy when Gaussian noise of variance 0.1: 0.966
3.   Test Accuracy when Gaussian noise of variance 1: 0.971


#### Data Augmentation: Vertical and horizontal flipping to the training and testing data. Gaussian noise of variance 0.01/0.1/1 to all images.

In [ ]:
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('A3E1_4.ipynb')

File ‘colab_pdf.py’ already there; not retrieving.

